In [1]:
import simpy
import numpy as np

# Parámetros globales
N = 3  # Número de ONUs
R = 10_000  # Tasa total de transmisión (Mbps)
T_cycle = 2  # Longitud del ciclo de polling (ms)

# Clase ONU
class ONU:
    def __init__(self, id, avg_rate):
        self.id = id
        self.avg_rate = avg_rate  # Tasa promedio de generación (paquetes/ms)
        self.ef_request = 0  # Solicitudes para EF
        self.af_request = 0  # Solicitudes para AF
        self.be_request = 0  # Solicitudes para BE
        self.ef_grant = 0  # Asignación EF
        self.af_grant = 0  # Asignación AF
        self.be_grant = 0  # Asignación BE

    def generate_requests(self):
        """Generar solicitudes acumuladas para el ciclo."""
        self.ef_request = np.random.poisson(self.avg_rate * 0.2)  # 20% EF
        self.af_request = np.random.poisson(self.avg_rate * 0.5)  # 50% AF
        self.be_request = np.random.poisson(self.avg_rate * 0.3)  # 30% BE

# Clase OLT
class OLT:
    def __init__(self, env, num_onus, cycle_time, total_bandwidth):
        self.env = env
        self.num_onus = num_onus
        self.cycle_time = cycle_time
        self.total_bandwidth = total_bandwidth
        self.onus = []  # Lista de ONUs

    def allocate_bandwidth(self):
        """Asignar ancho de banda basado en prioridades."""
        ef_total = sum(onu.ef_request for onu in self.onus)
        af_be_total = sum(onu.af_request + onu.be_request for onu in self.onus)

        # Asignación EF
        for onu in self.onus:
            if ef_total > 0:
                onu.ef_grant = (onu.ef_request / ef_total) * self.total_bandwidth * 0.6  # 60% para EF
            else:
                onu.ef_grant = 0

        # Asignación AF y BE (40% restante)
        remaining_bw = self.total_bandwidth - sum(onu.ef_grant for onu in self.onus)
        for onu in self.onus:
            if af_be_total > 0:
                onu.af_grant = (onu.af_request / af_be_total) * remaining_bw * 0.7  # 70% para AF
                onu.be_grant = (onu.be_request / af_be_total) * remaining_bw * 0.3  # 30% para BE
            else:
                onu.af_grant, onu.be_grant = 0, 0

    def run(self):
        """Ejecutar ciclos de polling."""
        while True:
            # Recolectar solicitudes y asignar ancho de banda
            for onu in self.onus:
                onu.generate_requests()
            self.allocate_bandwidth()
            
            # Imprimir resultados del ciclo
            print(f"Cycle {self.env.now}:")
            for onu in self.onus:
                print(f"ONU {onu.id}: EF={onu.ef_request}/{onu.ef_grant:.2f}, "
                      f"AF={onu.af_request}/{onu.af_grant:.2f}, "
                      f"BE={onu.be_request}/{onu.be_grant:.2f}")
            yield self.env.timeout(self.cycle_time)

# Crear entorno
env = simpy.Environment()
olt = OLT(env, N, T_cycle, R)

# Crear ONUs y agregarlas al OLT
for i in range(N):
    onu = ONU(i, avg_rate=100)  # Tasa promedio de generación (paquetes/ms)
    olt.onus.append(onu)

# Iniciar simulación
env.process(olt.run())
env.run(until=10)  # Simular 10 ms

Cycle 0:
ONU 0: EF=21/2290.91, AF=59/709.01, BE=22/113.30
ONU 1: EF=18/1963.64, AF=52/624.89, BE=22/113.30
ONU 2: EF=16/1745.45, AF=46/552.79, BE=32/164.81
Cycle 2:
ONU 0: EF=22/2315.79, AF=47/557.63, BE=21/106.78
ONU 1: EF=14/1473.68, AF=56/664.41, BE=30/152.54
ONU 2: EF=21/2210.53, AF=49/581.36, BE=33/167.80
Cycle 4:
ONU 0: EF=17/1888.89, AF=49/566.94, BE=39/193.39
ONU 1: EF=14/1555.56, AF=46/532.23, BE=33/163.64
ONU 2: EF=23/2555.56, AF=50/578.51, BE=25/123.97
Cycle 6:
ONU 0: EF=17/1888.89, AF=48/649.28, BE=20/115.94
ONU 1: EF=28/3111.11, AF=34/459.90, BE=26/150.72
ONU 2: EF=9/1000.00, AF=49/662.80, BE=30/173.91
Cycle 8:
ONU 0: EF=25/2343.75, AF=56/678.79, BE=24/124.68
ONU 1: EF=21/1968.75, AF=43/521.21, BE=27/140.26
ONU 2: EF=18/1687.50, AF=50/606.06, BE=31/161.04
